# This notebook is to be used for the Coursera capstone project.

In [1]:
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes
print('Folium & Geopy installed')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


import folium # plotting library


print('Libraries imported.')

Libraries imported.


In [3]:
# The code was removed by Watson Studio for sharing.

In [6]:
address = '453 Northampton St, Easton, PA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.69159985 -75.2124331441372


In [7]:
#Pizza place category ID = 4bf58dd8d48988d1ca941735
#Bar category ID         = 4bf58dd8d48988d116941735
search_cat_id = '4bf58dd8d48988d116941735,4bf58dd8d48988d1ca941735'
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, search_cat_id)
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,hierarchy,id,...,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '50327c8591d4c4b30a586d5d', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,4e308cb42271352f54404f58,...,196,"[542 Northampton St (at 6th St), Easton, PA 18...","[{'label': 'display', 'lat': 40.69080446448073...",40.690804,-75.214514,18042,PA,Two Rivers Brewing Co.,v-1574284513,68953243
1,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",287828,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/ginos-pizza...,False,NaN,4c2e2bf7987f20a123af4aaa,...,146,"[349 Northampton St, Easton, PA 18042, United ...","[{'label': 'display', 'lat': 40.69155300000000...",40.691553,-75.210703,18042,PA,Gino's Pizza,v-1574284513,NaN
2,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,[],57366a2ccd10997f565f5fb8,...,156,"[Easton, PA, United States]","[{'label': 'display', 'lat': 40.691547, 'lng':...",40.691547,-75.210580,NaN,PA,Scratch,v-1574284513,NaN
3,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,4d7313bd3815f04db7679767,...,128,"[15 N 4th St (Northampton), Easton, PA 18042, ...","[{'label': 'display', 'lat': 40.69136729080200...",40.691367,-75.210948,18042,PA,Lafayette Bar,v-1574284513,NaN
4,"[{'id': '4bf58dd8d48988d123941735', 'name': 'W...",NaN,NaN,NaN,NaN,NaN,NaN,False,[],5966a867a9e40247a77cedac,...,185,"[Easton, PA 18042, United States]","[{'label': 'display', 'lat': 40.690907, 'lng':...",40.690907,-75.210430,18042,PA,Tolino Wine Bar,v-1574284513,NaN


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Two Rivers Brewing Co.,Brewery,542 Northampton St,US,Easton,United States,at 6th St,196,"[542 Northampton St (at 6th St), Easton, PA 18...","[{'label': 'display', 'lat': 40.69080446448073...",40.690804,-75.214514,18042,PA,4e308cb42271352f54404f58
1,Gino's Pizza,Pizza Place,349 Northampton St,US,Easton,United States,NaN,146,"[349 Northampton St, Easton, PA 18042, United ...","[{'label': 'display', 'lat': 40.69155300000000...",40.691553,-75.210703,18042,PA,4c2e2bf7987f20a123af4aaa
2,Scratch,Pizza Place,NaN,US,Easton,United States,NaN,156,"[Easton, PA, United States]","[{'label': 'display', 'lat': 40.691547, 'lng':...",40.691547,-75.210580,NaN,PA,57366a2ccd10997f565f5fb8
3,Lafayette Bar,Bar,15 N 4th St,US,Easton,United States,Northampton,128,"[15 N 4th St (Northampton), Easton, PA 18042, ...","[{'label': 'display', 'lat': 40.69136729080200...",40.691367,-75.210948,18042,PA,4d7313bd3815f04db7679767
4,Tolino Wine Bar,Wine Bar,NaN,US,Easton,United States,NaN,185,"[Easton, PA 18042, United States]","[{'label': 'display', 'lat': 40.690907, 'lng':...",40.690907,-75.210430,18042,PA,5966a867a9e40247a77cedac
5,Latino Heat,Bar,NaN,US,Easton,United States,NaN,227,"[Easton, PA 18042, United States]","[{'label': 'display', 'lat': 40.69108944304095...",40.691089,-75.209820,18042,PA,4eebfb4c8231688f1cac68e2
6,Larry Holmes Round 1,Pub,NaN,US,NaN,United States,NaN,559,"[Pennsylvania, United States]","[{'label': 'display', 'lat': 40.6901131, 'lng'...",40.690113,-75.206102,NaN,Pennsylvania,4e74eee0fa76059700c74ddf
7,Stoke Coal Fire Pizza,Pizza Place,1 Lehns Ct,US,Easton,United States,Center Square,330,"[1 Lehns Ct (Center Square), Easton, PA 18042,...","[{'label': 'display', 'lat': 40.69090121142747...",40.690901,-75.208633,18042,PA,56aa9945498e8dd1c7bf66f5
8,Pearly Baker's Alehouse,Pub,11 Centre Sq,US,Easton,United States,NaN,327,"[11 Centre Sq, Easton, PA 18042, United States]","[{'label': 'display', 'lat': 40.69089263266136...",40.690893,-75.208668,18042,PA,4ace9d5af964a52070d120e3
9,The Standard,Bar,5 Centre Sq,US,Easton,United States,NaN,298,"[5 Centre Sq, Easton, PA 18042, United States]","[{'label': 'display', 'lat': 40.69112600636392...",40.691126,-75.208957,18042,PA,4b32d4f7f964a520b01425e3


In [9]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the State Theatre
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='State Theatre',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the bars in blue drink markers and pizza places in red heart markers
for lat, lng, label, cat in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name, dataframe_filtered.categories):
    label = folium.Popup(label, parse_html=True)
    if cat == "Pizza Place":
        icon_type = 'heart'
        color_type = 'red'
    else:
        icon_type = 'glass'
        color_type = 'blue'
    folium.Marker(
        [lat, lng],
        popup=label,
        icon=folium.Icon(icon=icon_type, color=color_type)
    ).add_to(venues_map)

# display map
venues_map